<a href="https://colab.research.google.com/github/IKeeso/Project9_data/blob/main/Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install stanza
!git clone https://github.com/IKeeso/Project9_data.git

fatal: destination path 'Project9_data' already exists and is not an empty directory.


In [10]:
import stanza
from stanza.utils.conll import CoNLL

!ls Project9_data


expressen.txt  Project.ipynb


In [5]:
stanza.download('sv')
nlp = stanza.Pipeline(lang='sv')

INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Downloading default packages for language: sv (Swedish) ...


INFO:stanza:Downloaded file to /root/stanza_resources/sv/default.zip
INFO:stanza:Finished downloading models and saved to /root/stanza_resources
INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Loading these models for language: sv (Swedish):
| Processor | Package            |
----------------------------------
| tokenize  | talbanken          |
| pos       | talbanken_charlm   |
| lemma     | talbanken_nocharlm |
| depparse  | talbanken_charlm   |
| ner       | suc3shuffle        |

INFO:stanza:Using device: cpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: pos
INFO:stanza:Loading: lemma
INFO:stanza:Loading: depparse
INFO:stanza:Loading: ner
INFO:stanza:Done loading processors!


In [21]:
def tag_and_convert(filename):
  with open(f"Project9_data/{filename}", 'r', encoding='utf-8') as file:
    text = file.read()
    doc = nlp(text)
    file.close()
    pathname = filename.split(".")[0]
  CoNLL.write_doc2conll(doc, f"Project9_data/{pathname}.conllu")


In [ ]:
tag_and_convert("expressen.txt")